### In deze notebook wordt de code en methode weergegeven mbt het maken van voorspellingen op de gefinetunede voorspellingen op de testdataset. De code is vrijwel identiek aan model-test.ipynb, waar verdere toelichtingen worden besproken.

## Valdetectiemodel setup

In [ ]:
from datetime import datetime
from urllib.request import urlopen

import numpy as np
import PIL.Image
import tensorflow.lite as tflite


MODEL_FILENAME = 'model.tflite'
LABELS_FILENAME = 'labels.txt'

od_model = None
labels = None


class ObjectDetection:
    OUTPUT_TENSOR_NAMES = ['detected_boxes', 'detected_scores', 'detected_classes']

    def __init__(self, model_filename):
        self._interpreter = tflite.Interpreter(model_path=model_filename)
        self._interpreter.allocate_tensors()

        input_details = self._interpreter.get_input_details()
        output_details = self._interpreter.get_output_details()
        assert len(input_details) == 1
        self._input_index = input_details[0]['index']

        output_name_index = {d['name']: d['index'] for d in output_details}
        self._output_indexes = [output_name_index[name] for name in self.OUTPUT_TENSOR_NAMES]

        self._input_size = int(input_details[0]['shape'][1])

    def predict_image(self, image):
        image = image.convert('RGB') if image.mode != 'RGB' else image
        image = image.resize((self._input_size, self._input_size))

        input_array = np.array(image, dtype=np.float32)[np.newaxis, :, :, :]
        self._interpreter.set_tensor(self._input_index, input_array)
        self._interpreter.invoke()

        outputs = [self._interpreter.get_tensor(i) for i in self._output_indexes]
        return outputs


def initialize():
    global od_model
    od_model = ObjectDetection(MODEL_FILENAME)
    global labels
    with open(LABELS_FILENAME) as f:
        labels = [label.strip() for label in f.readlines()]


def predict_url(image_url):
    with urlopen(image_url) as binary:
        image = PIL.Image.open(binary)
        return predict_image(image)


def predict_image(image):
    assert od_model is not None
    assert labels is not None

    predictions = od_model.predict_image(image)

    predictions = [{'probability': round(float(p[1]), 8),
                    'tagId': int(p[2]),
                    'tagName': labels[p[2]],
                    'boundingBox': {
                        'left': round(float(p[0][0]), 8),
                        'top': round(float(p[0][1]), 8),
                        'width': round(float(p[0][2] - p[0][0]), 8),
                        'height': round(float(p[0][3] - p[0][1]), 8)
                        }
                    } for p in zip(*predictions)]

    response = {'id': '', 'project': '', 'iteration': '', 'created': datetime.utcnow().isoformat(),
                'predictions': predictions}

    return response


## Logica voor het uitvoeren van de voorspellingen, inclusief het plaatsen van bounding boxen en het berekenen van de metrics.

In [ ]:
initialize()

import os
import time
import PIL.Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.pyplot as plt

def show_image_matplotlib(image, title=""):
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(title)
    plt.axis("off")
    plt.tight_layout()
    plt.show()

def draw_bounding_box_with_title(image, bbox, predicted_label, probability, title=""):
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    width, height = image.size
    left = bbox['left'] * width
    top = bbox['top'] * height
    box_width = bbox['width'] * width
    box_height = bbox['height'] * height

    rect = patches.Rectangle((left, top), box_width, box_height,
                             linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)

    label_text = f"{predicted_label} ({probability:.2f})"
    ax.text(left, top - 10, label_text, color='white', fontsize=12,
            backgroundcolor='red')

    ax.set_title(title, fontsize=14)
    plt.axis("off")
    plt.tight_layout()
    plt.show()


def predict_falls(base_path, THRESHOLD):
    start_time = time.time()
    total_predictions = 0
    rooms_to_predict = [7,8,11,15,21,31]

    TP = FP = TN = FN = 0

    POSE_LABELS = ["laying", "sitting", "standing", "bending"]

    def is_laying_true(filename):
        if "_not_laying" in filename:
            return False
        elif "_laying" in filename:
            return True

    def is_laying_pred(prediction):
        return (
            prediction['tagName'] == "laying"
            and prediction['probability'] >= THRESHOLD
        )

    for i in rooms_to_predict:
        room_path = os.path.join(base_path, f"room{i}")
        if not os.path.isdir(room_path):
            print(f"{room_path} bestaat niet, overslaan...")
            continue

        for filename in os.listdir(room_path):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue

            filepath = os.path.join(room_path, filename)
            image = PIL.Image.open(filepath)

            try:
                result = predict_image(image)
                total_predictions += 1
            except Exception as e:
                print(f"Fout bij voorspellen van {filepath}: {e}")
                continue

            predictions = result['predictions']
            if not predictions:
                print(f"Geen voorspelling voor {filename}")
                continue

            pose_predictions = [p for p in predictions if p['tagName'] in POSE_LABELS]

            if pose_predictions:
                best_pred = max(pose_predictions, key=lambda p: p['probability'])
                predicted = is_laying_pred(best_pred)
            else:
                predicted = False

            actual = is_laying_true(filename)

            if actual and predicted:
                TP += 1
            elif not actual and predicted:
                FP += 1
                print(f"[FALSE POSITIVE] Room: room{i}, File: {filename}")
                draw_bounding_box_with_title(
                    image,
                    bbox=best_pred['boundingBox'],
                    predicted_label=best_pred['tagName'],
                    probability=best_pred['probability'],
                    title=f"False Positive - room{i}/{filename}"
                )
            elif not actual and not predicted:
                TN += 1
            elif actual and not predicted:
                FN += 1
                print(f"[FALSE NEGATIVE] Room: room{i}, File: {filename}")
                draw_bounding_box_with_title(
                    image,
                    bbox=best_pred['boundingBox'],
                    predicted_label=best_pred['tagName'],
                    probability=best_pred['probability'],
                    title=f"False Negative - room{i}/{filename}"
                )

    end_time = time.time()
    total_time = end_time - start_time
    avg_time_per_prediction = total_time / total_predictions if total_predictions > 0 else 0

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    print("\n=== Resultaten ===")
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"\n=== Timing ===")
    print(f"Totale tijd: {total_time:.2f} seconden")
    print(f"Aantal voorspellingen: {total_predictions}")
    print(f"Gemiddelde tijd per voorspelling: {avg_time_per_prediction:.4f} seconden")


### Baseline

In [ ]:
predict_falls("data-filtered-rooms", 0.5)

### Traditionele methoden

In [ ]:
predict_falls("data-zerodce-lite", 0.05)

In [ ]:
predict_falls("data-gamma", 0.05)

### 50 epochs 1e-5

In [ ]:
predict_falls("data-zero-dce-finetuned50", 0.5)

### 100 epochs 1e-5

In [ ]:
predict_falls("data-zero-dce-finetuned100", 0.5)

### 50 epochs 1e-5 + 50 epochs 1e-100

In [ ]:
predict_falls("data-zero-dce-finetuned", 0.5)

### 100 epochs 1e-6

In [ ]:
predict_falls("data-zero-dce-finetuned1006", 0.5)

### 30 epochs 1e-7

In [ ]:
predict_falls("data-zero-dce-finetuned1007", 0.5)

### 100 epochs 1e-5, focus op shuffle in de batches

In [ ]:
predict_falls("data-zero-dce-finetuned-100-diffbatch", 0.5)